In [82]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

try:
    scope.dis()
    target.dis()
except Exception:
    pass
    

scope = cw.scope()
target = cw.target(scope)

scope.default_setup()

cw.program_target(scope, cw.programmers.STM32FProgrammer, "./search-CWNANO.hex")

scope.arm()

scope.adc.samples = 2000


See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13279 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13279 bytes


In [83]:
import time
import numpy as np

def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.simpleserial_write('s', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace(as_int=True)
    return np.array(trace)

In [84]:
%matplotlib tk

scope.adc.samples = 1000

ele = [128, 3, 255, 1, 63]
traces=[]
for i in ele:
    temp_trace = np.array(cap_pass_trace(bytearray([i])))
    traces.append(temp_trace)


fig, ax = plt.subplots(5)

for i in range(5):
    ax[i].plot(traces[i])


In [85]:
trc=[]
for i in range(256):
    temp_trace = np.array(cap_pass_trace(bytearray([i])))
    trc.append(temp_trace)
    

In [86]:
from numpy import genfromtxt
pattern = genfromtxt('pat_final.csv', delimiter=',')

In [87]:
# pattern = traces[2][275:289]
len_arr = len(pattern)

res = {}


for j in range(len(trc)):
    temp_list = []
    for i in range(len(trc[0])-len_arr):
        temp = trc[0+j][i:i+len_arr]
        cf = np.corrcoef(pattern,temp)[0][1]
        temp_list.append(cf)
    index = np.argmax(np.array(temp_list))
    a = res.get(index,[])
    a.append(j)
    res[index]=a
    print(f"j: {j}\t index: {index} \t cf: {temp_list[index]}")

j: 0	 index: 293 	 cf: 0.7611760153530409
j: 1	 index: 464 	 cf: 0.8169205072508577
j: 2	 index: 407 	 cf: 0.8137583025956564
j: 3	 index: 407 	 cf: 0.796373410099317
j: 4	 index: 407 	 cf: 0.8096144854724646
j: 5	 index: 407 	 cf: 0.7976640258185347
j: 6	 index: 293 	 cf: 0.7835652481558067
j: 7	 index: 350 	 cf: 0.8047378270478733
j: 8	 index: 122 	 cf: 0.768135665132969
j: 9	 index: 122 	 cf: 0.7646694257634358
j: 10	 index: 464 	 cf: 0.805128982287628
j: 11	 index: 65 	 cf: 0.782522966150356
j: 12	 index: 464 	 cf: 0.806474536204
j: 13	 index: 464 	 cf: 0.8032421949126393
j: 14	 index: 236 	 cf: 0.7734287567234879
j: 15	 index: 236 	 cf: 0.7962633980121319
j: 16	 index: 65 	 cf: 0.7543735029433473
j: 17	 index: 122 	 cf: 0.7633019384396955
j: 18	 index: 464 	 cf: 0.7725332325266693
j: 19	 index: 65 	 cf: 0.7666549161486697
j: 20	 index: 464 	 cf: 0.795180523635669
j: 21	 index: 464 	 cf: 0.7983804545984083
j: 22	 index: 65 	 cf: 0.7849685616328524
j: 23	 index: 65 	 cf: 0.782971271

j: 193	 index: 430 	 cf: 0.6476164835704383
j: 194	 index: 523 	 cf: 0.6381706859380355
j: 195	 index: 464 	 cf: 0.6556362578529198
j: 196	 index: 430 	 cf: 0.6327936104758418
j: 197	 index: 464 	 cf: 0.8064606983542245
j: 198	 index: 464 	 cf: 0.6394748743357337
j: 199	 index: 464 	 cf: 0.8201601698408127
j: 200	 index: 407 	 cf: 0.7914274282786082
j: 201	 index: 350 	 cf: 0.7122079001981395
j: 202	 index: 350 	 cf: 0.7690112347161754
j: 203	 index: 464 	 cf: 0.7053779964009723
j: 204	 index: 430 	 cf: 0.6494479486504404
j: 205	 index: 430 	 cf: 0.6608825374779226
j: 206	 index: 464 	 cf: 0.6848513560421085
j: 207	 index: 464 	 cf: 0.7922309511792945
j: 208	 index: 464 	 cf: 0.6364810596381175
j: 209	 index: 464 	 cf: 0.7766288651345933
j: 210	 index: 407 	 cf: 0.7848240192925345
j: 211	 index: 293 	 cf: 0.7215635847312766
j: 212	 index: 464 	 cf: 0.7521664189414956
j: 213	 index: 293 	 cf: 0.7192691137571713
j: 214	 index: 293 	 cf: 0.7382025621909057
j: 215	 index: 464 	 cf: 0.79640

In [88]:
for i in sorted(res.keys()):
    print(i ,res[i])

65 [11, 16, 19, 22, 23, 25, 27, 30, 34, 35, 36, 37, 39, 40, 43, 44, 45, 46, 47, 51, 52, 53, 56, 57, 58, 59, 61, 62, 64, 65, 66, 70, 71, 72, 73, 74, 78, 79, 80, 81, 82, 86, 87, 88, 89, 90, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]
122 [8, 9, 17, 24, 32, 33, 38, 41, 42, 48, 49, 50, 54, 55, 60, 128]
236 [14, 15, 31, 63, 162, 169, 170, 230, 233, 234, 235, 236, 237, 238, 239]
293 [0, 6, 159, 160, 175, 211, 213, 214, 224, 227, 229, 231, 243]
350 [7, 143, 163, 167, 178, 179, 187, 201, 202, 219, 220, 228, 232]
407 [2, 3, 4, 5, 76, 132, 135, 136, 137, 140, 145, 148, 149, 151, 153, 155, 156, 157, 161, 164, 173, 185, 189, 191, 200, 210, 216, 218, 226, 242, 252, 254]
430 [144, 168, 176, 184, 186, 192, 193, 196, 204, 205, 251]
448 [222]
464 [1, 10, 12, 13, 18, 20, 21, 26, 28, 29, 67, 68, 69, 75, 77, 83, 84, 85, 91, 92, 129, 130, 131, 133, 134, 138, 139, 141, 142, 146, 147, 1

In [80]:
correct_elts = bytearray([118, 59, 91, 152, 232, 207])

target.simpleserial_write('a', correct_elts)
res = target.simpleserial_read('r', 1)
print(res)

CWbytearray(b'01')


In [64]:
# np.savetxt("pat_final.csv", pattern, delimiter=",")

file_path = "counter/"
for i,val in enumerate([128, 3, 255, 1, 63]):
    np.savetxt(file_path + "sample_trace" + str(i)+".csv", traces[i], delimiter=",")


In [8]:
from numpy import genfromtxt

my_data = genfromtxt('pat.csv', delimiter=',')

In [9]:
my_data

array([ 64., 180., 206., 137., 132., 142., 186., 180., 141.,  60.,  67.])

In [ ]:
# [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 
#     25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 
#     47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 
#     70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 
#     92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 
#     112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132,
#     133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 
#     152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 
#     171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
#     191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 206, 207, 208, 209, 210, 211, 212,
#     213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 
#     233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 
#     253, 254, 255]

# # [113, 126, 52, 205, 204, 3]
# scope.dis()
# target.dis()

In [ ]:
ele = [0,128,255,204]
trc2=[]
for i in ele:
    temp_trace = np.array(cap_pass_trace(bytearray([i])))
    trc2.append(temp_trace)
    
fig, ax = plt.subplots(4)

for i in range(4):
    ax[i].plot(trc2[i])